## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks/air-quality-covid.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Identify available dates and temporal frequency of observations for a given collection - NO₂
   2. Pass the STAC item into raster API `/stac/tilejson.json` endpoint
   3. We'll visualize two tiles (side-by-side) allowing for comparison of each of the time points using `folium.plugins.DualMap`
   

## About the Data

This dataset is of monthly nitrogen dioxide [(NO₂) levels](https://radiantearth.github.io/stac-browser/#/external/staging-stac.delta-backend.com/collections/no2-monthly) values across the globe. Darker colors indicate higher NO₂ levels and more activity. Lighter colors indicate lower levels of NO₂ and less activity. Missing pixels indicate areas of no data most likely associated with cloud cover or snow.

## The Case Study - Air Quality and COVID-19

In this notebook, we'll walk through the development of side-by-side comparisons of NO₂ levels before and after government lockdowns as demonstrated `Seeing Rebounds in NO₂` in this VEDA Discovery story: [Air Quality and COVID-19 available on the VEDA Dashboard](https://www.earthdata.nasa.gov/dashboard/covid-19/discoveries/air-quality-and-covid-19). 

## Querying the STAC API

In [1]:
import requests
from folium import Map, TileLayer

In [2]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://staging-stac.delta-backend.com"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

# Declare collection of interest - Nitrogen Oxide
collection_name = "no2-monthly"

In [3]:
#Fetch STAC collection
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'no2-monthly',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly'}],
 'title': 'NO₂',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2016-01-01 00:00:00+00',
     '2022-12-01 00:00:00+00']]}},
 'license': 'MIT',
 'summaries': {'datetime': ['2016-01-01T00:00:00Z', '2022-12-01T00:00:00Z'],
  'cog_default': {'max': 50064805976866816, 'min': -10183824872833024}},
 'description': 'Darker colors indicate higher nitrogen dioxide (NO₂) levels and more activity. Lighter colors indicate lower levels of N

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2015 to December 2022. By looking at the `dashboard:time density` we observe that the periodic frequency of these observations is monthly. 

In [4]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=100").json()["features"]
print(f"Found {len(items)} items")

Found 84 items


This makes sense as there are 7 years between 2016 - 2022, with 12 months per year, meaning 84 records in total.  

Below, we'll use the `cog_default` values to provide our upper and lower bounds in `rescale_values`. 

In [5]:
rescale_values = collection["summaries"]["cog_default"]
rescale_values

{'max': 50064805976866816, 'min': -10183824872833024}

## Exploring Changes in Nitrogen Oxide (NO₂) Related to Changes in Human Behavior - Using the Raster API

We will explore changes in air quality due to changes in human behaviour resulting from the COVID-19 pandemic. With people largely confined to their homes to reduce the spread of the novel coronavirus, scientists were anticipated there were likely to be fewer cars, planes, and ships emitting fossil fuel pollution. In this notebook, we'll explore the impacts these government lockdowns had on specific air pollutants (i.e., NO₂) and explore these changes over time. We'll then visualize the outputs on a map using `folium`. 

In [6]:
# to access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:7]: item for item in items} 

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice times, once for February 2020 and again for February 2022, so that we can visualize each event independently. 

In [7]:
february_2020_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2020-02']['collection']}&item={items['2020-02']['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=cool"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
february_2020_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_0.10x0.10_202002_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=cool&rescale=-10183824872833024%2C50064805976866816'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

In [8]:
february_2022_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2022-02']['collection']}&item={items['2022-02']['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=cool"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
february_2022_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://staging-raster.delta-backend.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=no2-monthly&item=OMI_trno2_0.10x0.10_202202_Col3_V4.nc&assets=cog_default&color_formula=gamma+r+1.05&colormap_name=cool&rescale=-10183824872833024%2C50064805976866816'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.0, -90.0, 180.0, 90.0],
 'center': [0.0, 0.0, 0]}

## Seeing Rebounds in NO₂

Air pollutants with short lifespans, like NO₂, decreased dramatically with COVID-related shutdowns in the spring of 2020 (see lefthand side map). As the world began to re-open and mobility restrictions eased, travel increased and alongside it NO₂ pollutants. Air quality levels are now returning to pre-pandemic levels (see righthand side map). 

Scroll and zoom within the maps below, the side-by-side comparison will follow wherever you explore. Darker purples indicate higher NO₂ levels and more activity. Lighter blues indicate lower levels of NO₂ and less activity.

In [9]:
# We'll import folium to map and folium.plugins to allow mapping side-by-side
import folium
import folium.plugins

# Set initial zoom and map for NO2 Layer
m = folium.plugins.DualMap(location=(33.6901, 118.9325), zoom_start=5)

# February 2020
map_layer_2020 = TileLayer(
    tiles=february_2020_tile["tiles"][0],
    attr="VEDA",
    opacity=0.8,
)
map_layer_2020.add_to(m.m1)

# February 2022
map_layer_2022 = TileLayer(
    tiles=february_2022_tile["tiles"][0],
    attr="VEDA",
    opacity=0.8,
)
map_layer_2022.add_to(m.m2)

m



## Summary

In this case study we have successfully visualized how NASA monitors NO₂ emissions from space. By showcasing lockdown (February 2020) and post-lockdown (February 2022) snapshots of air quality side-by-side, we demonstrate how quickly atmospheric NO₂ responds to reductions in emissions and human behavior. 